In [23]:
#import the necessary libraries for dataset preparation, feature engineering, model training
from sklearn import model_selection, preprocessing, metrics, linear_model, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
from imblearn.under_sampling import (RandomUnderSampler, 
                                    NearMiss, 
                                    InstanceHardnessThreshold,
                                    CondensedNearestNeighbour,
                                    EditedNearestNeighbours,
                                    RepeatedEditedNearestNeighbours,
                                    AllKNN,
                                    NeighbourhoodCleaningRule,
                                    OneSidedSelection,
                                    TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import make_pipeline
import pandas as pd, numpy, string
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
#Remove Special Charactors
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

In [24]:
#Import Training and Testing Data
train = pd.read_csv(r"D:\Task\Analytics_Vidhya\Twitter_Sentiment_Analysis\train_E6oV3lV.csv")
print("Training Set:"% train.columns, train.shape, len(train))
test = pd.read_csv(r"D:\Task\Analytics_Vidhya\Twitter_Sentiment_Analysis\test_tweets_anuFYb8.csv")
print("Test Set:"% test.columns, test.shape, len(test))

Training Set: (31962, 3) 31962
Test Set: (17197, 2) 17197


In [25]:
pd.set_option('max_colwidth', None)

In [26]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation


In [27]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedication #willpower to find #newmaterialsâ¦
1,31964,@user #white #supremacists want everyone to see the new â #birdsâ #movie â and hereâs why
2,31965,safe ways to heal your #acne!! #altwaystoheal #healthy #healing!!
3,31966,"is the hp and the cursed child book up for reservations already? if yes, where? if no, when? ððð #harrypotter #pottermore #favorite"
4,31967,"3rd #bihday to my amazing, hilarious #nephew eli ahmir! uncle dave loves you and missesâ¦"


In [28]:
#Percentage of Positive/Negative
print("Positive: ", train.label.value_counts()[0]/len(train)*100,"%")
print("Negative: ", train.label.value_counts()[1]/len(train)*100,"%")

Positive:  92.98542018647143 %
Negative:  7.014579813528565 %


In [29]:
porter=PorterStemmer()
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    #Stemming
    stem_sentence=[]
    for word in words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    words="".join(stem_sentence).strip()
    return words
nums = [0,len(train)]
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    clean_tweet_texts.append(tweet_cleaner(train['tweet'][i]))
nums = [0,len(test)]
test_tweet_texts = []
for i in range(nums[0],nums[1]):
    test_tweet_texts.append(tweet_cleaner(test['tweet'][i])) 
train_clean = pd.DataFrame(clean_tweet_texts,columns=['tweet'])
train_clean['label'] = train.label
train_clean['id'] = train.id
test_clean = pd.DataFrame(test_tweet_texts,columns=['tweet'])
test_clean['id'] = test.id

In [30]:
train_clean.head()

,tweet,label,id
0,when a father is dysfunct and is so selfish he drag hi kid into hi dysfunct run,0,1
1,thank for lyft credit i can t use caus they don t offer wheelchair van in pdx disapoint getthank,0,2
2,bihday your majesti,0,3
3,model i love u take with u all the time in ur,0,4
4,factsguid societi now motiv,0,5


In [31]:
test_clean.head()

,tweet,id
0,studiolif aislif requir passion dedic willpow to find newmateri,31963
1,white supremacist want everyon to see the new bird movi and here s whi,31964
2,safe way to heal your acn altwaystoh healthi heal,31965
3,is the hp and the curs child book up for reserv alreadi if ye where if no when harrypott pottermor favorit,31966
4,rd bihday to my amaz hilari nephew eli ahmir uncl dave love you and miss,31967


In [32]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_clean['tweet'],train_clean['label'])

In [40]:
valid_x.shape

(7991,)

In [41]:
#split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_clean['tweet'],train_clean['label'])
#label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [47]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(train_clean['tweet'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [13]:
#Return the f1 Score
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    return metrics.f1_score(valid_y,predictions)

In [14]:
accuracyORIGINAL = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
accuracyORIGINAL = train_model(svm.LinearSVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM Baseline, WordLevel TFIDF: ", accuracyORIGINAL)


LR Baseline, WordLevel TFIDF:  0.570673712021136
SVM Baseline, WordLevel TFIDF:  0.7187141216991965


Smote

In [54]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
sm_xtrain_tfidf, sm_train_y = smote.fit_resample(xtrain_tfidf,train_y)
accuracySMOTE = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),sm_xtrain_tfidf, sm_train_y, xvalid_tfidf)
print ("LR SMOTE, WordLevel TFIDF: ", accuracySMOTE)
accuracySMOTE = train_model(svm.LinearSVC(),sm_xtrain_tfidf, sm_train_y, xvalid_tfidf)
print ("SVC SMOTE, WordLevel TFIDF: ", accuracySMOTE)


LR SMOTE, WordLevel TFIDF:  0.6733167082294265
SVC SMOTE, WordLevel TFIDF:  0.6900269541778976


In [65]:
X_train = train_clean['tweet']

In [66]:
target_train =train_clean['label']

In [67]:
Y_test = test_clean['tweet']

In [68]:
#label encode the target variable 
encoder = preprocessing.LabelEncoder()
target_train = encoder.fit_transform(target_train)


In [69]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(train_clean['tweet'])
xtrain_tfidf =  tfidf_vect.transform(X_train)


In [70]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(test_clean['tweet'])
xvalid_tfidf =  tfidf_vect.transform(Y_test)

In [73]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
sm_xtrain_tfidf, sm_train_y = smote.fit_resample(xtrain_tfidf,target_train)


In [80]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(sm_xtrain_tfidf, sm_train_y)
print (clf.intercept_, clf.coef_)

[-1.57655263] [[ 1.29537209 -0.23438079 -0.17431581 ... -0.02814457 -0.00331777
  -0.02575562]]


In [81]:
predictions = clf.predict(xvalid_tfidf)

ValueError: X has 21312 features per sample; expecting 31427